# 8.4 Parsing with Context-Free Grammar

In [1]:
import nltk
#S can be broken down to NP, followed by VP
#VP can be broken down to V followed by NP, or V followed by NP followed by PP
#PP can be broken down to P followed by NP
grammar1 = nltk.CFG.fromstring("""
  S -> NP VP              
  VP -> V NP | V NP PP    
  PP -> P NP              
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

### Recursive Descent Parser

In [2]:
rd_parser = nltk.RecursiveDescentParser(grammar1, trace=2) #Trace = 2 provides the full parsing steps
sent = "Mary saw a dog".split()
for t in rd_parser.parse(sent):
    print(t)
    print()
#Observations: 
#1. The parser actually reads the grammar and parses the sentence according to that.
#2. It starts with S -> NP, VP followed by NP -> John etc.

#The second part demonstrates all the valid sentences with that vocabulary

Parsing 'Mary saw a dog'
    [ * S ]
  E [ * NP VP ]
  E [ * 'John' VP ]
  E [ * 'Mary' VP ]
  M [ 'Mary' * VP ]
  E [ 'Mary' * V NP ]
  E [ 'Mary' * 'saw' NP ]
  M [ 'Mary' 'saw' * NP ]
  E [ 'Mary' 'saw' * 'John' ]
  E [ 'Mary' 'saw' * 'Mary' ]
  E [ 'Mary' 'saw' * 'Bob' ]
  E [ 'Mary' 'saw' * Det N ]
  E [ 'Mary' 'saw' * 'a' N ]
  M [ 'Mary' 'saw' 'a' * N ]
  E [ 'Mary' 'saw' 'a' * 'man' ]
  E [ 'Mary' 'saw' 'a' * 'dog' ]
  M [ 'Mary' 'saw' 'a' 'dog' ]
  + [ 'Mary' 'saw' 'a' 'dog' ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))

  E [ 'Mary' 'saw' 'a' * 'cat' ]
  E [ 'Mary' 'saw' 'a' * 'telescope' ]
  E [ 'Mary' 'saw' 'a' * 'park' ]
  E [ 'Mary' 'saw' * 'an' N ]
  E [ 'Mary' 'saw' * 'the' N ]
  E [ 'Mary' 'saw' * 'my' N ]
  E [ 'Mary' 'saw' * Det N PP ]
  E [ 'Mary' 'saw' * 'a' N PP ]
  M [ 'Mary' 'saw' 'a' * N PP ]
  E [ 'Mary' 'saw' 'a' * 'man' PP ]
  E [ 'Mary' 'saw' 'a' * 'dog' PP ]
  M [ 'Mary' 'saw' 'a' 'dog' * PP ]
  E [ 'Mary' 'saw' 'a' 'dog' * P NP ]
  E [ 'Mary' 'saw' 'a

### Shift Reduce Parser

In [3]:
#Shift-Reduce Parser
sr_parser = nltk.ShiftReduceParser(grammar1, trace=2)
for t in sr_parser.parse(sent):
    print(t)
#From here you can see shift steps and reduce steps very clearly, with the * as the separator

Parsing 'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


### Well-formed Substring Tables (WFST)

In [4]:
#Init_Wfst function
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

In [5]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1, nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                              (start, nt1, mid, nt2, end, start, index[(nt1, nt2)], end))
    return wfst

In [6]:
def display(wfst, tokens):
    print('\nWFST  ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d     " % i, end = " ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or "."), end =" ")
        print()

1. Given a sentence $a_1, \cdots, a_n$ with $n$ words, instantiate a triangular matrix $A$ where $A$ has $n$ rows and $n$ columns. The rows are labelled $\left\{ 0, \cdots, n-1 \right\}$ and the columns are labelled $\left\{ 1, \cdots, n \right\}$

2. Assign the cells in $A$ for each word $a_i,i \in [1, \cdots, n]$ where the horizontal axis represents the start position of the string and the vertical axis represents the end position of the string. The cell will have the value of **the word category of the word*.

Example 1:

Given the sentence `'I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas'`, the representation of the word `shot` in $A$ appears in position $(1, 2)$. Since `shot` belongs to the category `v`, the cell $(1, 2)$ contains the value `v`. This way, for every word in the sentence, we can look up which grammar category it belongs to.

In [7]:
from nltk import CFG
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
 """)
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST  1    2    3    4    5    6    7   
0      NP   .    .    .    .    .    .    
1      .    V    .    .    .    .    .    
2      .    .    Det  .    .    .    .    
3      .    .    .    N    .    .    .    
4      .    .    .    .    P    .    .    
5      .    .    .    .    .    Det  .    
6      .    .    .    .    .    .    N    


We can fill in a value in $A_{(i, j)}$ if there is a production $A \rightarrow B C$ and we find a non-terminal (not-at-the-end) $B$ in $(i, k)$ and $C$ in $(k, j)$. For example, $NP \rightarrow Det\space N$ exists in the grammar. Hence, we can put $NP$ in cell $(2, 4)$. An $NP$ exists in the starting position of $2$ and ends at $4$.

In [8]:
#Given the same logic, there is a production of 
#VP -> V NP resulting in VP in (1, 4) and
#S -> NP VP resulting in S in (0, 4) and so on and so forth.
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST  1    2    3    4    5    6    7   
0      NP   .    .    S    .    .    S    
1      .    V    .    VP   .    .    VP   
2      .    .    Det  NP   .    .    .    
3      .    .    .    N    .    .    .    
4      .    .    .    .    P    .    PP   
5      .    .    .    .    .    Det  NP   
6      .    .    .    .    .    .    N    


In [9]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
